In [48]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Reshape
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec, KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
data = pd.read_csv("Drive/GPT-wiki-intro.csv")
data

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


In [50]:
data.drop(['url','title_len','wiki_intro_len','generated_intro_len','prompt','generated_text','prompt_tokens','generated_text_tokens'],axis=1,inplace=True)
print(data)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                               wiki_intro  \
0       Sexhow railway station was a railway station b...   
1       In Finnish folklore, all places and things, an...   
2       In mathematics, specifically differential calc...   
3       is a Japanese shōjo manga series written and i...   
4       Robert Milner "Rob" Bradley, Jr. (born August ...   
...                                                   ...   
149995  Randy Borum 

In [52]:
human_texts=data.copy()
human_texts.drop(['generated_intro'],axis=1,inplace=True)
human_texts=human_texts.tail(150000)
human_texts.rename(columns = {'wiki_intro':'text'}, inplace = True)
result=[]
for i in range(150000):
  result.append("0")
human_texts['result']=result
print(human_texts)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                                     text result  
0       Sexhow railway station was a railway station b...      0  
1       In Finnish folklore, all places and things, an...      0  
2       In mathematics, specifically differential calc...      0  
3       is a Japanese shōjo manga series written and i...      0  
4       Robert Milner "Rob" Bradley, Jr. (born August ...      0  
...                                          

In [53]:
ai_texts=data.copy()
ai_texts.drop(['wiki_intro'],axis=1,inplace=True)
ai_texts=ai_texts.tail(150000)
ai_texts.rename(columns = {'generated_intro':'text'}, inplace = True)
result=[]
for i in range(150000):
  result.append("1")
ai_texts['result']=result
print(ai_texts)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                                     text result  
0       Sexhow railway station was a railway station l...      1  
1       In Finnish folklore, all places and things, an...      1  
2       In mathematics, specifically differential calc...      1  
3       is a Japanese shōjo manga series written and i...      1  
4       Robert Milner "Rob" Bradley, Jr. (born August ...      1  
...                                          

In [54]:
all_texts = pd.concat([human_texts,ai_texts])
print(all_texts)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                                     text result  
0       Sexhow railway station was a railway station b...      0  
1       In Finnish folklore, all places and things, an...      0  
2       In mathematics, specifically differential calc...      0  
3       is a Japanese shōjo manga series written and i...      0  
4       Robert Milner "Rob" Bradley, Jr. (born August ...      0  
...                                          

In [55]:
texts = all_texts['text']

In [56]:
# Pre-process text data
def pre_process_text(text):
    # Lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    # Tokenize text and remove stop words
    tokens = nltk.word_tokenize(text)
    stop_words = nltk.corpus.stopwords.words("english")
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    # Join tokens back into text
    text = " ".join(lemmatized_tokens)
    return text

In [57]:
processed_texts = texts.apply(pre_process_text)

In [58]:
all_texts['text'] = processed_texts
all_texts.to_csv("all_texts.csv")
print(all_texts)

              id                     title  \
0       63064638    Sexhow railway station   
1         279621                  Etiäinen   
2         287229  Inverse function theorem   
3       26712375         Stepping on Roses   
4       38894426               Rob Bradley   
...          ...                       ...   
149995  44173767               Randy Borum   
149996  33564134           Sa'och language   
149997   4219548           Philip Hanawalt   
149998   2625970         Vossius Gymnasium   
149999  32345102           Simone Stratigo   

                                                     text result  
0       sexhow railway station railway station built s...      0  
1       finnish folklor place thing also human be halt...      0  
2       mathemat specif differenti calculu invers func...      0  
3       japanes shōjo manga seri written illustr rinko...      0  
4       robert milner rob bradley jr born august 24 19...      0  
...                                          